<a href="https://colab.research.google.com/github/Yixuan-042/CASA0025/blob/main/notebooks/W04_week4_postgis2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [21]:
# %pip install duckdb leafmap lonboard
import duckdb
import leafmap

In [20]:
%pip install duckdb duckdb-engine jupysql

In [22]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [23]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [24]:
%sql duckdb:///:memory:
# %sql duckdb:///path/to/file.db

In [25]:
%%sql

SELECT * FROM duckdb_extensions();

,extension_name,loaded,installed,install_path,description,aliases,extension_version,install_mode,installed_from
0,arrow,False,False,,A zero-copy data integration between Apache Ar...,[],,None,
1,autocomplete,False,False,,Adds support for autocomplete in the shell,[],,None,
2,aws,False,False,,Provides features that depend on the AWS SDK,[],,None,
3,azure,False,False,,Adds a filesystem abstraction for Azure blob s...,[],,None,
4,delta,False,False,,Adds support for Delta Lake,[],,None,
5,excel,False,False,,Adds support for Excel-like format strings,[],,None,
6,fts,False,True,(BUILT-IN),Adds support for Full-Text Search Indexes,[],,STATICALLY_LINKED,
7,httpfs,False,False,,Adds support for reading and writing files ove...,"[http, https, s3]",,None,
8,iceberg,False,False,,Adds support for Apache Iceberg,[],,None,
9,icu,True,True,(BUILT-IN),Adds support for time zones and collations usi...,[],,STATICALLY_LINKED,


In [26]:
%%sql

INSTALL httpfs;
LOAD httpfs;

,Success


In [28]:
import os
import zipfile
import requests

# 下载 ZIP 文件
url = "https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip"
zip_path = "nyc_data.zip"
extract_path = "nyc_data"

# 下载文件
response = requests.get(url)
with open(zip_path, "wb") as f:
    f.write(response.content)

# 解压 ZIP 文件
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Files extracted to: {extract_path}")


Files extracted to: nyc_data


In [30]:
import os

# 检查解压目录中的文件
extracted_files = os.listdir(extract_path)
print(f"Files in '{extract_path}': {extracted_files}")


Files in 'nyc_data': ['nyc_streets.shp', 'nyc_streets.dbf', 'nyc_census_blocks.dbf', 'nyc_census_blocks.shp', 'nyc_homicides.dbf', 'nyc_neighborhoods.shx', 'nyc_subway_stations.prj', 'nyc_neighborhoods.prj', 'nyc_subway_stations.dbf', 'nyc_streets.prj', 'nyc_census_blocks.shx', 'nyc_census_sociodata.sql', 'nyc_homicides.shx', 'README.txt', 'nyc_homicides.prj', 'nyc_census_blocks.prj', 'nyc_neighborhoods.shp', 'nyc_subway_stations.shp', 'nyc_subway_stations.shx', 'nyc_homicides.shp', 'nyc_streets.shx', 'nyc_neighborhoods.dbf']


In [31]:
pip install geopandas


In [32]:
import geopandas as gpd

# 定义文件路径
data_dir = "nyc_data/"

# 读取各个 Shapefile 文件
census_blocks = gpd.read_file(f"{data_dir}nyc_census_blocks.shp")
homicides = gpd.read_file(f"{data_dir}nyc_homicides.shp")
neighborhoods = gpd.read_file(f"{data_dir}nyc_neighborhoods.shp")
streets = gpd.read_file(f"{data_dir}nyc_streets.shp")
subway_stations = gpd.read_file(f"{data_dir}nyc_subway_stations.shp")

# 查看数据
print("Census Blocks:")
print(census_blocks.head())

print("\nHomicides:")
print(homicides.head())

print("\nNeighborhoods:")
print(neighborhoods.head())

print("\nStreets:")
print(streets.head())

print("\nSubway Stations:")
print(subway_stations.head())


Census Blocks:
             BLKID  POPN_TOTAL  POPN_WHITE  POPN_BLACK  POPN_NATIV  \
0  360850009001000          97          51          32           1   
1  360850020011000          66          52           2           0   
2  360850040001000          62          14          18           2   
3  360850074001000         137          92          12           0   
4  360850096011000         289         230           0           0   

   POPN_ASIAN  POPN_OTHER       BORONAME  \
0           5           8  Staten Island   
1           7           5  Staten Island   
2          25           3  Staten Island   
3          13          20  Staten Island   
4          32          27  Staten Island   

                                            geometry  
0  POLYGON ((577856.547 4499583.235, 577862.635 4...  
1  POLYGON ((578620.717 4495974.818, 578535.358 4...  
2  POLYGON ((577227.224 4495995.067, 577155.625 4...  
3  POLYGON ((579037.033 4494421.77, 579000.015 44...  
4  POLYGON ((577652.483 

In [33]:
# 将 GeoDataFrame 保存为 CSV
census_blocks.to_csv(f"{data_dir}nyc_census_blocks.csv", index=False)
homicides.to_csv(f"{data_dir}nyc_homicides.csv", index=False)
neighborhoods.to_csv(f"{data_dir}nyc_neighborhoods.csv", index=False)
streets.to_csv(f"{data_dir}nyc_streets.csv", index=False)
subway_stations.to_csv(f"{data_dir}nyc_subway_stations.csv", index=False)

print("Files converted to CSV.")


Files converted to CSV.


In [34]:
import duckdb

# 创建 DuckDB 数据库连接
conn = duckdb.connect("nyc_data.duckdb")

# 定义 CSV 文件路径
datasets = {
    "nyc_census_blocks": f"{data_dir}nyc_census_blocks.csv",
    "nyc_homicides": f"{data_dir}nyc_homicides.csv",
    "nyc_neighborhoods": f"{data_dir}nyc_neighborhoods.csv",
    "nyc_streets": f"{data_dir}nyc_streets.csv",
    "nyc_subway_stations": f"{data_dir}nyc_subway_stations.csv",
}

# 加载 CSV 文件到 DuckDB 表
for table_name, file_path in datasets.items():
    print(f"Loading file: {file_path}")
    conn.execute(f"""
        CREATE TABLE {table_name} AS
        SELECT * FROM read_csv_auto('{file_path}');
    """)
    print(f"Table {table_name} created successfully.")

# 检查表是否成功创建
tables = conn.execute("SHOW TABLES;").fetchall()
print("Tables in database:", tables)

# 查询示例
result = conn.execute("SELECT * FROM nyc_neighborhoods LIMIT 5;").fetchdf()
print(result)


Loading file: nyc_data/nyc_census_blocks.csv
Table nyc_census_blocks created successfully.
Loading file: nyc_data/nyc_homicides.csv
Table nyc_homicides created successfully.
Loading file: nyc_data/nyc_neighborhoods.csv
Table nyc_neighborhoods created successfully.
Loading file: nyc_data/nyc_streets.csv
Table nyc_streets created successfully.
Loading file: nyc_data/nyc_subway_stations.csv
Table nyc_subway_stations created successfully.
Tables in database: [('nyc_census_blocks',), ('nyc_homicides',), ('nyc_neighborhoods',), ('nyc_streets',), ('nyc_subway_stations',)]
    BORONAME                      NAME  \
0   Brooklyn               Bensonhurst   
1  Manhattan              East Village   
2  Manhattan              West Village   
3  The Bronx              Throggs Neck   
4  The Bronx  Wakefield-Williamsbridge   

                                            geometry  
0  POLYGON ((582771.4257198056 4495167.427365481,...  
1  POLYGON ((585508.7534890148 4509691.267208001,...  
2  POLYGON

In [36]:
import duckdb

# 连接到数据库
conn = duckdb.connect("nyc_data.duckdb")

# 查看所有表
tables = conn.execute("SHOW TABLES;").fetchall()
print("Available tables:", tables)


Available tables: [('nyc_census_blocks',), ('nyc_homicides',), ('nyc_neighborhoods',), ('nyc_streets',), ('nyc_subway_stations',)]


Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

1. **What subway station is in \'Little Italy\'? What subway route is it on?**

2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

4. **What neighborhood has the highest population density (persons/km2)?**


When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other. This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

In [ ]:
%pip install duckdb duckdb-engine jupysql

In [ ]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [ ]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

You can set a spatial index on each of these tables as follows:

`CREATE INDEX index_name ON table_name USING RTREE(geom);`

## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.